# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
# CONFIG = or_suite.envs.env_configs.resource_allocation_foodbank_config(5)
CONFIG = or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
#  'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
 'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[1.98  1.979 2.007 1.943 1.991 2.071 1.984 2.002 2.083 1.956]
 [2.994 3.01  2.993 2.941 3.014 2.976 3.046 3.006 3.    3.06 ]
 [3.99  3.957 3.929 4.106 3.957 3.988 4.023 3.978 3.991 3.967]] [[0.9996   0.932559 1.048951 0.937751 0.960919 1.081959 0.997744 0.949996
  1.076111 0.962064]
 [2.109964 1.9739   1.920951 1.879519 2.085804 2.001424 2.147884 1.807964
  2.106    2.1904  ]
 [3.0639   3.025151 2.989959 3.012764 2.803151 2.897856 3.034471 2.897516
  2.848919 2.967911]]
Mean and variance endomwnets:
[[1.971 1.991 2.01  2.002 2.037 1.984 1.996 1.976 2.014 1.985]
 [3.018 2.97  2.933 3.042 3.02  2.981 2.974 3.001 3.048 3.057]
 [3.926 3.981 4.033 4.046 3.912 4.045 3.928 4.032 4.014 4.094]] [[0.954159 0.942919 1.0879   0.953996 0.989631 1.011744 0.945984 0.999424
  1.019804 1.030775]
 [2.059676 2.0671   1.886511 2.064236 2.1316   2.064639 2.095324 2.134999
  2.195696 1.985751]
 [3.034524 3.074639 2.967911 2.703884 3.146256 3.138975 2.946816 2.838976
  2.979804

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG), \
                      'Budget' : lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5
Lower and Upper Solutions:
[[ 0.       0.13359]
 [-0.       0.13359]
 [ 0.16428  0.     ]]
[[0.      0.19537]
 [0.      0.19537]
 [0.24026 0.     ]]
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[ 0.       0.13324]
 [-0.       0.13324]
 [ 0.16294  0.     ]]
[[0.      0.26711]
 [0.      0.26711]
 [0.32665 0.     ]]
Writing to file data.csv
Guardrail-0.25
Lower and Upper Solutions:
[[0.      0.13232]
 [0.      0.13232]
 [0.16241 0.     ]]
[[0.      0.30234]
 [0.      0.30234]
 [0.37109 0.     ]]
Writing to file data.csv
        Algorithm    Reward      Time     Space  Efficiency  Hindsight Envy  \
0   Guardrail-0.5 -7.180845  5.202982 -14853.74 -220.793910     -156.601470   
1   Guardrail-0.3 -7.153558  5.205476 -12642.07 -175.641999     -224.420292   
2  Guardrail-0.25 -7.340392  5.199175 -12535.07 -177.091640     -256.127386   

   Counterfactual Envy  Budget  
0           -44.538440     0.0  
1           -75.136680     0.0  
2           -93.111919   

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)